In [2]:
import os

In [51]:
%pwd

'c:\\Users\\koush\\OneDrive\\Desktop\\mouna'

In [54]:
os.chdir('../')

In [55]:
%pwd

'C:\\Users\\koush\\OneDrive\\Desktop\\mouna\\Text-Summarizer-Project'

In [56]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    s3_bucket_name: str
    input_key: str
    output_key: str
    local_data_file: Path


In [57]:

import sys
from textsummarizer.config_constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [58]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "src"))

from textsummarizer.config_constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

print(CONFIG_FILE_PATH, PARAMS_FILE_PATH)  # test



config\config.yaml params.yaml


In [59]:

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(
            root_dir=config.root_dir,
            s3_bucket_name=config.s3_bucket_name,
            input_key=config.input_key,
            output_key=config.output_key,
            local_data_file=config.local_data_file
        )


In [45]:
!pip install boto3
!pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 10.8/10.8 MB 67.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---------------------------------------- 14.9/14.9 MB 84.9 MB/s eta 0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)


In [82]:
!pip install datasets

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached huggingface_hub-0.34.4-py3-none-any.whl (561 kB)
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ------------------------------ --------- 19.1/25.2 MB 93.0 MB/s eta 0:00:01
   ---------------------------------------- 25.2/25.2 MB 69.5 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)


In [ ]:
import os
from pathlib import Path
from datasets import load_dataset
from textsummarizer.logging import logger

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def load_dataset_from_s3(self):
        """
        Load Hugging Face dataset directly from S3
        """
        dataset = load_dataset(
            "arrow",
            data_files={
                "train": f"s3://{self.config.s3_bucket_name}/samsum_dataset/train/data-00000-of-00001.arrow",
                "validation": f"s3://{self.config.s3_bucket_name}/samsum_dataset/validation/data-00000-of-00001.arrow",
                "test": f"s3://{self.config.s3_bucket_name}/samsum_dataset/test/data-00000-of-00001.arrow"
            })
        logger.info(f"Dataset loaded with splits: {dataset.keys()}")
        return dataset

    def save_local(self, dataset):
        """
        Save dataset locally in Arrow format (keeps splits)
        """
        local_dir = Path(self.config.root_dir)
        local_dir.mkdir(parents=True, exist_ok=True)
        dataset.save_to_disk(str(local_dir))
        logger.info(f"Dataset saved locally at {local_dir}")


In [61]:
%pwd

'C:\\Users\\koush\\OneDrive\\Desktop\\mouna\\Text-Summarizer-Project'

In [88]:
!pip install s3fs

   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------  13.1/13.3 MB 82.3 MB/s eta 0:00:01
   ---------------------------------------- 13.3/13.3 MB 64.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.38
    Uninstalling botocore-1.37.38:
      Successfully uninstalled botocore-1.37.38


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.37.38 requires botocore<1.38.0,>=1.37.38, but you have botocore 1.37.3 which is incompatible.
datasets 3.1.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.
s3transfer 0.11.5 requires botocore<2.0a.0,>=1.37.4, but you have botocore 1.37.3 which is incompatible.


In [89]:

config_manager = ConfigurationManager()
data_ingestion_config = config_manager.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)

# Load dataset from S3
dataset = data_ingestion.load_dataset_from_s3()

# Save locally
data_ingestion.save_local(dataset)

# Optional: Access a sample
print(dataset["train"][0])

[2025-09-07 19:36:50,405: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-07 19:36:50,410: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-07 19:36:50,413: INFO: common: created directory at: artifacts]
[2025-09-07 19:36:50,416: INFO: common: created directory at: artifacts/data_ingestion]


Generating train split: 14732 examples [00:00, 645270.80 examples/s]
Generating validation split: 818 examples [00:00, 137799.85 examples/s]
Generating test split: 819 examples [00:00, 135449.51 examples/s]


[2025-09-07 19:37:03,779: INFO: 284089137: Dataset loaded with splits: dict_keys(['train', 'validation', 'test'])]


Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 62554.81 examples/s]

[2025-09-07 19:37:03,898: INFO: 284089137: Dataset saved locally at artifacts\data_ingestion]
{'id': '13818513', 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)", 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}
